# Bang-bang

In [6]:
import time
import numpy as np

from coppeliasim_zmqremoteapi_client import RemoteAPIClient 

try:
    # Connect to the CoppeliaSim server
    client = RemoteAPIClient()
    sim = client.require("sim")
    sim.setStepping(True)

    # Handle para o ROBÔ
    robotname = 'Manta'
    robotHandle = sim.getObject('/' + robotname)
    
    # Handle para Direção e Motor
    steerHandle = sim.getObject('/' + robotname + '/steer_joint')
    motorHandle = sim.getObject('/' + robotname + '/motor_joint')
    
    # Configurações do Robô
    motor_torque = 60
    sim.setJointTargetForce(motorHandle, motor_torque)
    
    motor_velocity = 12
    sim.setJointTargetVelocity(motorHandle, motor_velocity)
  
    max_steer = np.deg2rad(10)
        
    # Lembrar de habilitar o 'Real-time mode'    
    # Parar a simulação se estiver executando
    initial_sim_state = sim.getSimulationState()
    if initial_sim_state != 0:
        sim.stopSimulation()
        time.sleep(1)
    
    # Inicia a simulação
    sim.startSimulation()
    sim.step()
    
    robotPos = sim.getObjectPosition(robotHandle, sim.handle_world)
    print('Pos: ', robotPos)
    
    print("Starting robot control loop...")
    while (sim_time := sim.getSimulationTime()) <= 25:
        print(f"Simulation time: {sim_time:.2f} [s]")
        
        dt = sim.getSimulationTimeStep()
        
        robotPos = sim.getObjectPosition(robotHandle, sim.handle_world)
       
        # Entrada de controle
        u = max_steer
        if robotPos[1] > 0:
            u = -max_steer
            
        sim.setJointTargetPosition(steerHandle, u)

        sim.step()

    # Parando o robô
    print("Stopping robot...")
    sim.setJointTargetVelocity(motorHandle, 0)

except Exception as e:
    print(f"An error occurred: {e}")
    
# Parando a simulação
sim.stopSimulation()

print('Program ended')

Pos:  [-11.00067265776295, 1.0000138842031245, 0.20782592977026582]
Starting robot control loop...
Simulation time: 0.05 [s]
Simulation time: 0.10 [s]
Simulation time: 0.15 [s]
Simulation time: 0.20 [s]
Simulation time: 0.25 [s]
Simulation time: 0.30 [s]
Simulation time: 0.35 [s]
Simulation time: 0.40 [s]
Simulation time: 0.45 [s]
Simulation time: 0.50 [s]
Simulation time: 0.55 [s]
Simulation time: 0.60 [s]
Simulation time: 0.65 [s]
Simulation time: 0.70 [s]
Simulation time: 0.75 [s]
Simulation time: 0.80 [s]
Simulation time: 0.85 [s]
Simulation time: 0.90 [s]
Simulation time: 0.95 [s]
Simulation time: 1.00 [s]
Simulation time: 1.05 [s]
Simulation time: 1.10 [s]
Simulation time: 1.15 [s]
Simulation time: 1.20 [s]
Simulation time: 1.25 [s]
Simulation time: 1.30 [s]
Simulation time: 1.35 [s]
Simulation time: 1.40 [s]
Simulation time: 1.45 [s]
Simulation time: 1.50 [s]
Simulation time: 1.55 [s]
Simulation time: 1.60 [s]
Simulation time: 1.65 [s]
Simulation time: 1.70 [s]
Simulation time: 

Simulation time: 15.45 [s]
Simulation time: 15.50 [s]
Simulation time: 15.55 [s]
Simulation time: 15.60 [s]
Simulation time: 15.65 [s]
Simulation time: 15.70 [s]
Simulation time: 15.75 [s]
Simulation time: 15.80 [s]
Simulation time: 15.85 [s]
Simulation time: 15.90 [s]
Simulation time: 15.95 [s]
Simulation time: 16.00 [s]
Simulation time: 16.05 [s]
Simulation time: 16.10 [s]
Simulation time: 16.15 [s]
Simulation time: 16.20 [s]
Simulation time: 16.25 [s]
Simulation time: 16.30 [s]
Simulation time: 16.35 [s]
Simulation time: 16.40 [s]
Simulation time: 16.45 [s]
Simulation time: 16.50 [s]
Simulation time: 16.55 [s]
Simulation time: 16.60 [s]
Simulation time: 16.65 [s]
Simulation time: 16.70 [s]
Simulation time: 16.75 [s]
Simulation time: 16.80 [s]
Simulation time: 16.85 [s]
Simulation time: 16.90 [s]
Simulation time: 16.95 [s]
Simulation time: 17.00 [s]
Simulation time: 17.05 [s]
Simulation time: 17.10 [s]
Simulation time: 17.15 [s]
Simulation time: 17.20 [s]
Simulation time: 17.25 [s]
S

# PID

In [34]:
import time
import numpy as np

from coppeliasim_zmqremoteapi_client import RemoteAPIClient 

try:
    # Connect to the CoppeliaSim server
    client = RemoteAPIClient()
    sim = client.require("sim")
    sim.setStepping(True)

    # Handle para o ROBÔ
    robotname = 'Manta'
    robotHandle = sim.getObject('/' + robotname)
    
    # Handle para Direção e Motor
    steerHandle = sim.getObject('/' + robotname + '/steer_joint')
    motorHandle = sim.getObject('/' + robotname + '/motor_joint')
    
    # Configurações do Robô
    motor_torque = 60
    sim.setJointTargetForce(motorHandle, motor_torque)
    
    motor_velocity = 12
    sim.setJointTargetVelocity(motorHandle, motor_velocity)
  
    max_steer = np.deg2rad(10)
        
    # Lembrar de habilitar o 'Real-time mode'    
    # Parar a simulação se estiver executando
    initial_sim_state = sim.getSimulationState()
    if initial_sim_state != 0:
        sim.stopSimulation()
        time.sleep(1)
    
    # Inicia a simulação
    sim.startSimulation()
    sim.step()
    
    robotPos = sim.getObjectPosition(robotHandle, sim.handle_world)
    print('Pos: ', robotPos)
    
    # Estado objetivo é y = 0
    setpoint = 0
    
    error = 0
    lastError = 0
    lastTime = 0
    cumError = 0
    
    print("Starting robot control loop...")
    while (sim_time := sim.getSimulationTime()) <= 20:
        print(f"Simulation time: {sim_time:.2f} [s]")
        
        dt = sim.getSimulationTimeStep()
        print(dt)
        
        robotPos = sim.getObjectPosition(robotHandle, sim.handle_world)
        
        # Erro     
        error = setpoint - robotPos[1]
        dError = (error - lastError) / dt
        cumError += error * dt
        
        # Ganhos básicos: kp = .4, kd = .5, ki = .01
        
        # Proporcional
        kp = .4
        up = kp*error
        
        # Derivativo
        kd = 0.5
        ud = kd*dError
        
        # Integral
        ki = .01
        ui = ki*cumError
    
        # Controller
        u = up + ud + ui
        
        print("===> u1: ", u)
    
        # Limitando o valor para +/- max
        u = max(min(u, max_steer), -max_steer)
        
        print("===> u2: ", u)
            
        sim.setJointTargetPosition(steerHandle, u)
        
        lastError = error

        sim.step()

    # Parando o robô
    print("Stopping robot...")
    sim.setJointTargetVelocity(motorHandle, 0)

except Exception as e:
    print(f"An error occurred: {e}")
    
# Parando a simulação
sim.stopSimulation()

print('Program ended')

Pos:  [-11.000671953514413, 1.0000104170222543, 0.20782273693172243]
Starting robot control loop...
Simulation time: 0.05 [s]
0.05
===> u1:  -10.400608342239956
===> u2:  -0.17453292519943295
Simulation time: 0.10 [s]
0.05
===> u1:  -0.4008253983000249
===> u2:  -0.17453292519943295
Simulation time: 0.15 [s]
0.05
===> u1:  -0.40309008249944817
===> u2:  -0.17453292519943295
Simulation time: 0.20 [s]
0.05
===> u1:  -0.4050311805714121
===> u2:  -0.17453292519943295
Simulation time: 0.25 [s]
0.05
===> u1:  -0.40508457545158044
===> u2:  -0.17453292519943295
Simulation time: 0.30 [s]
0.05
===> u1:  -0.4006733707944238
===> u2:  -0.17453292519943295
Simulation time: 0.35 [s]
0.05
===> u1:  -0.3952444075332478
===> u2:  -0.17453292519943295
Simulation time: 0.40 [s]
0.05
===> u1:  -0.39161898677383783
===> u2:  -0.17453292519943295
Simulation time: 0.45 [s]
0.05
===> u1:  -0.386602934911532
===> u2:  -0.17453292519943295
Simulation time: 0.50 [s]
0.05
===> u1:  -0.37946899961728114
===> u2:

Simulation time: 4.45 [s]
0.05
===> u1:  0.04077937146903365
===> u2:  0.04077937146903365
Simulation time: 4.50 [s]
0.05
===> u1:  0.03944667903634799
===> u2:  0.03944667903634799
Simulation time: 4.55 [s]
0.05
===> u1:  0.038166111552899305
===> u2:  0.038166111552899305
Simulation time: 4.60 [s]
0.05
===> u1:  0.03723188633192574
===> u2:  0.03723188633192574
Simulation time: 4.65 [s]
0.05
===> u1:  0.036778327625036966
===> u2:  0.036778327625036966
Simulation time: 4.70 [s]
0.05
===> u1:  0.035714949318941466
===> u2:  0.035714949318941466
Simulation time: 4.75 [s]
0.05
===> u1:  0.03484426700660824
===> u2:  0.03484426700660824
Simulation time: 4.80 [s]
0.05
===> u1:  0.03390006289191523
===> u2:  0.03390006289191523
Simulation time: 4.85 [s]
0.05
===> u1:  0.0331254724311606
===> u2:  0.0331254724311606
Simulation time: 4.90 [s]
0.05
===> u1:  0.03218917154390137
===> u2:  0.03218917154390137
Simulation time: 4.95 [s]
0.05
===> u1:  0.03151708492602481
===> u2:  0.0315170849260

Simulation time: 9.10 [s]
0.05
===> u1:  0.01610257535132402
===> u2:  0.01610257535132402
Simulation time: 9.15 [s]
0.05
===> u1:  0.016744671173393372
===> u2:  0.016744671173393372
Simulation time: 9.20 [s]
0.05
===> u1:  0.017507941925892796
===> u2:  0.017507941925892796
Simulation time: 9.25 [s]
0.05
===> u1:  0.018073474905728904
===> u2:  0.018073474905728904
Simulation time: 9.30 [s]
0.05
===> u1:  0.0187137831063926
===> u2:  0.0187137831063926
Simulation time: 9.35 [s]
0.05
===> u1:  0.01906128693397955
===> u2:  0.01906128693397955
Simulation time: 9.40 [s]
0.05
===> u1:  0.019756848364846497
===> u2:  0.019756848364846497
Simulation time: 9.45 [s]
0.05
===> u1:  0.020443284023667456
===> u2:  0.020443284023667456
Simulation time: 9.50 [s]
0.05
===> u1:  0.020833702899338856
===> u2:  0.020833702899338856
Simulation time: 9.55 [s]
0.05
===> u1:  0.02118010411369817
===> u2:  0.02118010411369817
Simulation time: 9.60 [s]
0.05
===> u1:  0.02136801827501338
===> u2:  0.0213680

Simulation time: 13.70 [s]
0.05
===> u1:  0.01677966659027219
===> u2:  0.01677966659027219
Simulation time: 13.75 [s]
0.05
===> u1:  0.01734961449238638
===> u2:  0.01734961449238638
Simulation time: 13.80 [s]
0.05
===> u1:  0.017300541252847287
===> u2:  0.017300541252847287
Simulation time: 13.85 [s]
0.05
===> u1:  0.01704590096803049
===> u2:  0.01704590096803049
Simulation time: 13.90 [s]
0.05
===> u1:  0.016820353828013057
===> u2:  0.016820353828013057
Simulation time: 13.95 [s]
0.05
===> u1:  0.01661897563055672
===> u2:  0.01661897563055672
Simulation time: 14.00 [s]
0.05
===> u1:  0.0165450305904392
===> u2:  0.0165450305904392
Simulation time: 14.05 [s]
0.05
===> u1:  0.0166078326175575
===> u2:  0.0166078326175575
Simulation time: 14.10 [s]
0.05
===> u1:  0.01691208386231253
===> u2:  0.01691208386231253
Simulation time: 14.15 [s]
0.05
===> u1:  0.01662142742874359
===> u2:  0.01662142742874359
Simulation time: 14.20 [s]
0.05
===> u1:  0.016003774672960925
===> u2:  0.01600

Simulation time: 18.15 [s]
0.05
===> u1:  0.00556642092094007
===> u2:  0.00556642092094007
Simulation time: 18.20 [s]
0.05
===> u1:  0.005071781167268703
===> u2:  0.005071781167268703
Simulation time: 18.25 [s]
0.05
===> u1:  0.004482204396495432
===> u2:  0.004482204396495432
Simulation time: 18.30 [s]
0.05
===> u1:  0.0041526169995083975
===> u2:  0.0041526169995083975
Simulation time: 18.35 [s]
0.05
===> u1:  0.003994562016456062
===> u2:  0.003994562016456062
Simulation time: 18.40 [s]
0.05
===> u1:  0.004586112785597094
===> u2:  0.004586112785597094
Simulation time: 18.45 [s]
0.05
===> u1:  0.00439656601919482
===> u2:  0.00439656601919482
Simulation time: 18.50 [s]
0.05
===> u1:  0.004171960183816367
===> u2:  0.004171960183816367
Simulation time: 18.55 [s]
0.05
===> u1:  0.0037837654422558688
===> u2:  0.0037837654422558688
Simulation time: 18.60 [s]
0.05
===> u1:  0.00369555249387786
===> u2:  0.00369555249387786
Simulation time: 18.65 [s]
0.05
===> u1:  0.003661567442378941

Simulation time: 22.55 [s]
0.05
===> u1:  0.007507219371183177
===> u2:  0.007507219371183177
Simulation time: 22.60 [s]
0.05
===> u1:  0.005811478057535301
===> u2:  0.005811478057535301
Simulation time: 22.65 [s]
0.05
===> u1:  0.0056828365191422345
===> u2:  0.0056828365191422345
Simulation time: 22.70 [s]
0.05
===> u1:  0.005706578059247527
===> u2:  0.005706578059247527
Simulation time: 22.75 [s]
0.05
===> u1:  0.0057141972229820415
===> u2:  0.0057141972229820415
Simulation time: 22.80 [s]
0.05
===> u1:  0.005749891214345888
===> u2:  0.005749891214345888
Simulation time: 22.85 [s]
0.05
===> u1:  0.005765037021269183
===> u2:  0.005765037021269183
Simulation time: 22.90 [s]
0.05
===> u1:  0.0058148745969663195
===> u2:  0.0058148745969663195
Simulation time: 22.95 [s]
0.05
===> u1:  0.0048790140479061245
===> u2:  0.0048790140479061245
Simulation time: 23.00 [s]
0.05
===> u1:  0.005080431712562188
===> u2:  0.005080431712562188
Simulation time: 23.05 [s]
0.05
===> u1:  0.00496847

# Para praticar

- Altere os ganhos e verifique o impacto de cada um no resultado final.